In [2]:
# def entropy
# def variance
# def info_gain
# def categorical_split
# def max_info (choose one of the unique value in one feature to have the highest info_gain => then choose the highest ig among the features)

In [3]:
import numpy as np
import pandas as pd 

data = pd.read_csv("500_Person_Gender_Height_Weight_Index.csv")
data

,Gender,Height,Weight,Index
0,Male,174,96,4
1,Male,189,87,2
2,Female,185,110,4
3,Female,195,104,3
4,Male,149,61,3
...,...,...,...,...
495,Female,150,153,5
496,Female,184,121,4
497,Female,141,136,5
498,Male,150,95,5


In [4]:
data['Obese'] = (data['Index'] >= 4).astype('int')
data.drop('Index', axis = 1, inplace= True)
data.head()

,Gender,Height,Weight,Obese
0,Male,174,96,1
1,Male,189,87,0
2,Female,185,110,1
3,Female,195,104,0
4,Male,149,61,0


'''
  Given a Pandas Series, it calculates the entropy. 
  y: variable with which calculate entropy.
  '''

In [5]:
def entropy(y):
    if isinstance (y, pd.Series):
        prob = y.value_counts()/y.shape[0]
        entropy = np.sum(-prob* np.log2(prob))
        return entropy
    
    else:
        raise ('Object must be a Pandas series')


entropy(data.Gender)

In [6]:
entropy(data.Gender)

0.9997114417528099

In [7]:
def variance(y):
    if len(y) == 1:
        return 0
    else : 
        return y.var()

In [8]:
def info_gain(y, mask, func = entropy):
    a = sum(mask)
    b = mask.shape[0] - a 

    if a == 0 or b == 0:
         return 0
    else:
        if y.dtypes != 'O': #if y is numeric
          return func(y) - (func(y[mask])*a/(a+b) + func(y[-mask])*b/(a+b))
        else: 
          return variance(y) - variance(y[mask])*a/(a+b) - variance(y[mask])*b/(a+b)

In [9]:
df = data['Obese'][data['Gender'] == 'Male']
print(df)
entropy(df)

0      1
1      0
4      0
5      0
6      1
      ..
487    0
488    1
494    1
498    1
499    1
Name: Obese, Length: 245, dtype: int32


0.9155506778147324

In [10]:
info_gain(data['Obese'], data['Gender'] == 'Male')


0.0005506911187602714

In [11]:
#dont know https://anderfernandez.com/en/blog/code-decision-tree-python-from-scratch/
import itertools
def categorical_split(a):
    a = a.unique()
    opciones = []

    for L in range (0, len(a) + 1):
        for subset in itertools.combinations(a, L):
            subset = list(subset)
            opciones.append(subset)
    
    return opciones[1:-1]


In [12]:
print(data['Gender'].unique())
print(categorical_split(data['Gender']))
#print(list(itertools.combinations(data['Gender'], 1)))

['Male' 'Female']
[['Male'], ['Female']]


In [13]:
def max_info_gain(x, y, func = entropy):
    ig = 0
    split_value = 0
    num = 0

    is_numeric = True if x.dtypes != 'O' else False

    #creating options according to variable types
    if is_numeric:
        options = x.sort_values().unique()[1:] #not starting by the first element since there's nothing to split before the first element
    else:
        options = categorical_split(x)
    
    #calculating information gain for different value
    for value in options:
        mask = x < value if is_numeric == True else x.isin(value) #if (value is in x)
        value_ig = info_gain(y, mask, func)

        if value_ig > ig:
            ig = value_ig
            split_value = value
            num = num + 1
    
    if num == 0:
        return (None, None, None, False)

    else: 
        return (ig, split_value, is_numeric, True)


In [14]:
#test
weight_ig, weight_split, _, _ = max_info_gain(data['Weight'], data['Obese'],)
print(weight_ig)
print(weight_split)

0.3824541370911897
103


In [15]:
#calculate the Information Gain for each of the predictor variables of the model.
masks = data.drop('Obese', axis = 1).apply(max_info_gain, y = data['Obese'])
print(masks)
masks.loc[3,:]
masks.loc[:, masks.loc[3, :]]
max(masks.loc[:, masks.loc[3, :]])

     Gender    Height    Weight
0  0.000551  0.064748  0.382454
1    [Male]       174       103
2     False      True      True
3      True      True      True


'Weight'

In [16]:
#highest Information Gain is Weight
#include the different hyperparameters that a decision tree generally offers, thus avoiding overfitting:
    #max_depth: maximum depth of the tree. If we set it to None, the tree will 
              # grow until all the leaves are pure or the hyperparameter min_samples_split has been reached.
    #min_samples_split: indicates the minimum number of observations a sheet must have to continue creating new nodes.
    #min_information_gain: the minimum amount the Information Gain must increase for the tree to continue growing.

In [17]:
#y: name of the target features
def get_best_split(y, data):
    masks = data.drop(y, axis = 1).apply(max_info_gain, y = data[y])
    if sum(masks.loc[3,:]) == 0:
        return (None, None, None, False)

    else:
        #choose only masks that can be splitted
        masks = masks.loc[:, masks.loc[3,:]] # choose all columns that has the 3th index True box
        split_vari = max(masks) #choose split feature
        split_ig = masks[split_vari][0]
        split_val = masks[split_vari][1]
        split_numeric = masks[split_vari][2]
    return (split_vari, split_val, split_ig, split_numeric)
        

In [18]:
get_best_split('Obese', data)

('Weight', 103, 0.3824541370911897, True)

In [19]:
'''
  Given a data and a split conditions, do the split.
  variable: variable with which make the split.
  value: value of the variable to make the split.
  data: data to be splitted.
  is_numeric: boolean considering if the variable to be splitted is numeric or not.
  '''
def make_split(variable, value, data, is_numeric):
    if is_numeric:
      data1 = data[data[variable] < value]
      data2 = data[(data[variable] < value) == False]
    else:
      data1 = data[data[variable].isin(value)]
      data2 = data[data[variable].isin(value) == False]
    return data1, data2


In [20]:
#Example/ Demonstration of the previous code
value = 'Weight'
print(type(value))
print(data[value] < 103)
data1 = data[data['Weight'] < 103] 
print(data1)

<class 'str'>
0       True
1       True
2      False
3      False
4       True
       ...  
495    False
496    False
497    False
498     True
499    False
Name: Weight, Length: 500, dtype: bool
     Gender  Height  Weight  Obese
0      Male     174      96      1
1      Male     189      87      0
4      Male     149      61      0
6      Male     147      92      1
8      Male     174      90      0
..      ...     ...     ...    ...
490  Female     164      59      0
492  Female     198      50      0
493  Female     170      53      0
494    Male     152      98      1
498    Male     150      95      1

[229 rows x 4 columns]


In [21]:
def make_pred(data, target_factor):
    '''
  Given the target variable, make a prediction.
  data: pandas series for target variable
  target_factor: boolean considering if the variable is a factor or not
  '''
    if target_factor:
        pred = data.value_counts().idxmax() #return which value has more occurence, in this case, there an "1" or "0"
    else:
        pred = data.mean()
    return pred


In [22]:
def train_tree(data, y, target_factor, max_depth = None, min_samples_split = None, min_information_gain = 1e-20, max_categories = 20, counter = 0):
    '''
  Trains a Decission Tree
  data: Data to be used to train the Decission Tree
  y: target variable column name
  target_factor: boolean to consider if target variable is factor or numeric.
  max_depth: maximum depth to stop splitting.
  min_samples_split: minimum number of observations to make a split.
  min_information_gain: minimum ig gain to consider a split to be valid.
  max_categories: maximum number of different values accepted for categorical values. High number of values will slow down learning process. R
'''
    # Check that max_categories is fulfilled
    if counter ==0:
        types = data.dtypes
        check_columns = types[types == "object"].index #deo hieu
        for columns in check_columns:
            var_length = len(data[columns].value_counts())
            if var_length > max_categories:
                raise ValueError('The variable ' + columns + ' has '+ str(var_length) + ' unique values, which is more than the accepted ones: ' +  str(max_categories))
    #check for depth condition
    if max_depth == None:
        dep_cond = True
    else:
        if counter < max_depth:
            dep_cond = True
        else: 
            dep_cond = False
    #check for sample condition
    if min_samples_split == None:
        sample_cond = True
    else:
        if data.shape[0] > min_samples_split: #number of rows
            sample_cond = True
        else: 
            sample_cond = False
    # Check for ig condition
    if dep_cond & sample_cond:
        var,val, ig, var_type = get_best_split(y, data)
        print(val)
        # If ig condition is fulfilled, make split
        if ig is not None and ig >= min_information_gain:
            counter = counter + 1
            left, right = make_split(var, val, data, var_type) #2 sub datasets

            #instantiate sub-tree (deo hieu)
            split_type = "<=" if var_type else "in"
            question = "{} {}  {}".format(var, split_type, val)
            subtree = {question: []} #[] represents values of 2 branches, {} represent one such branch 
    
    # Find answers (recursion)
            yes_ans = train_tree(left, y, target_factor, max_depth, min_samples_split, min_information_gain, max_categories, counter)
            no_ans = train_tree(right, y, target_factor, max_depth, min_samples_split, min_information_gain, max_categories, counter)
            print(yes_ans, no_ans)
            if yes_ans == no_ans: #neu cung class het thi ko chia ra
                subtree = yes_ans
            else:
                subtree[question].append(yes_ans)
                subtree[question].append(no_ans)
        #If it doesn't match IG condition, make prediction
        else:
            pred = make_pred(data[y], target_factor)
            return pred
    else:
        pred = make_pred(data[y], target_factor)
        return pred

    return subtree


In [23]:
max_depth = 5
min_samples_split = 20
min_information_gain = 1e-5

decisiones = train_tree(data, 'Obese', True, max_depth, min_samples_split, min_information_gain)

decisiones
#                W <= 103
#                 /   \
#           W<=66      1
#          /     \
#         0     W <=84
#                /    \
#            W<=74     W <=98
#           /  \         /  \
#        0   W <=75     1    0
#             /  \
#            1    0

103
66
None
84
74
72
0 0
75
1 0
0 {'Weight <=  75': [1, 0]}
98
97
1 1
1 0
{'Weight <=  74': [0, {'Weight <=  75': [1, 0]}]} {'Weight <=  98': [1, 0]}
0 {'Weight <=  84': [{'Weight <=  74': [0, {'Weight <=  75': [1, 0]}]}, {'Weight <=  98': [1, 0]}]}
116
104
107
108
1 1
1 1
1 1
None
1 1
{'Weight <=  66': [0, {'Weight <=  84': [{'Weight <=  74': [0, {'Weight <=  75': [1, 0]}]}, {'Weight <=  98': [1, 0]}]}]} 1


{'Weight <=  103': [{'Weight <=  66': [0,
    {'Weight <=  84': [{'Weight <=  74': [0, {'Weight <=  75': [1, 0]}]},
      {'Weight <=  98': [1, 0]}]}]},
  1]}

In [24]:
colname_d3 = ['x1', 'x2', 'y']
df_d3 = pd.read_csv('D3leaves.txt', names= colname_d3, sep = ' ')
temp = df_d3['y'] 
cu = df_d3.drop('y', axis = 1).apply(max_info_gain, y = temp)
cu = cu.loc[:, cu.loc[3, :]]
train_tree(df_d3,'y', True, max_depth, min_samples_split, min_information_gain)

1

In [25]:
colname_d3 = ['x1', 'x2', 'y']
df_d1 = pd.read_csv('D1.txt', names = colname_d3, sep = " ")
train_tree(df_d1,'y', True, max_depth, min_samples_split, min_information_gain)

0.201829
None
None
0 1


{'x2 <=  0.201829': [0, 1]}

In [26]:
df_d2 = pd.read_csv('D2.txt', names = colname_d3, sep = " ")
train_tree(df_d2,'y', True, max_depth, min_samples_split, min_information_gain)

0.506048
0.224236
0.053292
0.037708
None
0 0
0.053702
0.061886
0 0
1 0
0 {'x2 <=  0.053702': [1, 0]}
0.383738
0.379696
0.235379
1 0
{'x2 <=  0.235379': [1, 0]} 0
0.388699
0.390262
0 0
1 0
{'x2 <=  0.379696': [{'x2 <=  0.235379': [1, 0]}, 0]} {'x2 <=  0.388699': [1, 0]}
{'x2 <=  0.053292': [0, {'x2 <=  0.053702': [1, 0]}]} {'x2 <=  0.383738': [{'x2 <=  0.379696': [{'x2 <=  0.235379': [1, 0]}, 0]}, {'x2 <=  0.388699': [1, 0]}]}
0.88635
0.641964
0.516515
0.635533
1 0
1 {'x2 <=  0.635533': [1, 0]}
0.654331
0.658212
0 1
1 {'x2 <=  0.658212': [0, 1]}
{'x2 <=  0.516515': [1, {'x2 <=  0.635533': [1, 0]}]} {'x2 <=  0.654331': [1, {'x2 <=  0.658212': [0, 1]}]}
0.919236
0.917219
0.903656
1 1
1 0
None
{'x2 <=  0.917219': [1, 0]} 1
{'x2 <=  0.641964': [{'x2 <=  0.516515': [1, {'x2 <=  0.635533': [1, 0]}]}, {'x2 <=  0.654331': [1, {'x2 <=  0.658212': [0, 1]}]}]} {'x2 <=  0.919236': [{'x2 <=  0.917219': [1, 0]}, 1]}
{'x2 <=  0.224236': [{'x2 <=  0.053292': [0, {'x2 <=  0.053702': [1, 0]}]}, {'x2 <=  

{'x2 <=  0.506048': [{'x2 <=  0.224236': [{'x2 <=  0.053292': [0,
      {'x2 <=  0.053702': [1, 0]}]},
    {'x2 <=  0.383738': [{'x2 <=  0.379696': [{'x2 <=  0.235379': [1, 0]}, 0]},
      {'x2 <=  0.388699': [1, 0]}]}]},
  {'x2 <=  0.88635': [{'x2 <=  0.641964': [{'x2 <=  0.516515': [1,
        {'x2 <=  0.635533': [1, 0]}]},
      {'x2 <=  0.654331': [1, {'x2 <=  0.658212': [0, 1]}]}]},
    {'x2 <=  0.919236': [{'x2 <=  0.917219': [1, 0]}, 1]}]}]}

In [27]:
dbig = pd.read_csv('Dbig.txt', sep= " ", names= colname_d3)

np.random.seed(0)
ind_list = np.random.permutation(10000) #list of 10000 numbers in random numbers

training_ind = ind_list[:8192]
test_ind = ind_list[8192:]
train_set = dbig.loc[training_ind]
test_set = dbig.loc[test_ind]
print(train_set)
print(train_set['y'])

            x1        x2  y
9394  1.319820  1.151578  1
898  -1.222178  0.073686  1
2398 -0.759529  0.921726  0
5906  0.304684 -1.454185  1
2343 -0.775965  0.980504  0
...        ...       ... ..
4104 -0.244451 -1.227159  1
5989  0.329880 -1.459166  1
6537  0.492965 -0.955107  1
540  -1.332802  0.142872  1
80   -1.476813  0.281231  1

[8192 rows x 3 columns]
9394    1
898     1
2398    0
5906    1
2343    0
       ..
4104    1
5989    1
6537    1
540     1
80      1
Name: y, Length: 8192, dtype: int64


In [29]:
train_tree(train_set, 'y', True, max_depth, min_samples_split, min_information_gain)

-0.711517
-0.926187
-0.954911
None
-0.954629
None
0 1
1 {'x2 <=  -0.954629': [0, 1]}
-0.856344
-0.925588
-0.863642
1 1
0 1
-0.853943
-0.778137
1 1
0 1
{'x2 <=  -0.925588': [0, 1]} {'x2 <=  -0.853943': [0, 1]}
{'x2 <=  -0.954911': [1, {'x2 <=  -0.954629': [0, 1]}]} {'x2 <=  -0.856344': [{'x2 <=  -0.925588': [0, 1]}, {'x2 <=  -0.853943': [0, 1]}]}
1.232245
-0.221321
-0.500252
-0.50362
1 1
-0.46443
0 1
1 {'x2 <=  -0.46443': [0, 1]}
1.060339
0.107964
0 0
1.198584
0 1
0 {'x2 <=  1.198584': [0, 1]}
{'x2 <=  -0.500252': [1, {'x2 <=  -0.46443': [0, 1]}]} {'x2 <=  1.060339': [0, {'x2 <=  1.198584': [0, 1]}]}
None
{'x2 <=  -0.221321': [{'x2 <=  -0.500252': [1, {'x2 <=  -0.46443': [0, 1]}]}, {'x2 <=  1.060339': [0, {'x2 <=  1.198584': [0, 1]}]}]} 1
{'x2 <=  -0.926187': [{'x2 <=  -0.954911': [1, {'x2 <=  -0.954629': [0, 1]}]}, {'x2 <=  -0.856344': [{'x2 <=  -0.925588': [0, 1]}, {'x2 <=  -0.853943': [0, 1]}]}]} {'x2 <=  1.232245': [{'x2 <=  -0.221321': [{'x2 <=  -0.500252': [1, {'x2 <=  -0.46443': 

{'x2 <=  -0.711517': [{'x2 <=  -0.926187': [{'x2 <=  -0.954911': [1,
      {'x2 <=  -0.954629': [0, 1]}]},
    {'x2 <=  -0.856344': [{'x2 <=  -0.925588': [0, 1]},
      {'x2 <=  -0.853943': [0, 1]}]}]},
  {'x2 <=  1.232245': [{'x2 <=  -0.221321': [{'x2 <=  -0.500252': [1,
        {'x2 <=  -0.46443': [0, 1]}]},
      {'x2 <=  1.060339': [0, {'x2 <=  1.198584': [0, 1]}]}]},
    1]}]}